<a href="https://colab.research.google.com/github/raynardj/python4ml/blob/master/experiments/rl/atari_pooyan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pooyan with DQN

In [0]:
%matplotlib inline

Install modules

In [2]:
!apt-get install -y xvfb

!pip install pyvirtualdisplay
!pip install piglet

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

!pip install gym
!pip install gym[atari]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 9 not upgraded.
Need to get 783 kB of archives.
After this operation, 2,266 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.2 [783 kB]
Fetched 783 kB in 1s (999 kB/s)
Selecting previously unselected package xvfb.
(Reading database ... 130791 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.2_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.2) ...
Setting up xvfb (2:1.19.6-1ubuntu4.2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
  Stored in directory: /root/.cache/pip/wheels/d1/8c/16/1c64227974ae29c687e4cc30fd691d5c0

Import libraries and modules

In [0]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image


import torch
from torch import nn,optim
from torch.nn import functional as F
from torchvision import transforms as T

## Set up environment

In [4]:
env = gym.make("Pooyan-v0").unwrapped

# set up matplotlib
is_ipy = 'inline' in matplotlib.get_backend()
if is_ipy:
    from IPython import display
    
plt.ion()

# if GPU is used
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("We are using IPython", is_ipy)
print("We are on device", dev)

We are using IPython True
We are on device cuda


## Replay Memory

In [0]:
Transition = namedtuple("Transition",
                        ("state","action","next_state","reward"))

class Replay_Memory(object):
    def __init__(self,capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0
        
    def push(self,*args):
        """
        args: in order: state, action, next_state, reward
        """
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position+1) % self.capacity
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
        
    def __len__(self):
        return len(self.memory)

## DQN Algorithm

In [0]:
class DQN(nn.Module):
    def __init__(self, action_space = 2, flatten = 448):
        super(DQN,self).__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size=5,stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16,32, kernel_size=5,stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5,stride=2)
        self.bn3 = nn.BatchNorm2d(32)
        
        self.head = nn.Linear(flatten, action_space, )
        
    def forward(self,x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.view(x.size(0),-1)
        x = self.head(x)
        return x